In [1]:
import os
import sys
import pickle

import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
graph_n20_path = '/data/pately/graph_instance_evolution/saved_training_samples_mix_6000_20nodes/'
graph_n100_path = '/data/pately/graph_instance_evolution/saved_training_samples_mix_6000_100nodes/'

In [ ]:
# def read_graphs_in_path(path, extension='pickle'):
#     # print(path)
#     list_of_graphs = []
#     if os.path.isdir(path):
        
#         for filename in os.listdir(path):
#             print(filename)
#             file_path = os.path.join(path, filename)
#             # print(file_path)
#             if os.path.isfile(file_path):    
#                 if extension is not None:
#                     if filename.endswith(extension):
#                         dictionary = pickle.load(open(file_path, 'rb'))
#                         list_of_graphs.append(dictionary['graph'])
#                 else:
#                     list_of_graphs.append(filename)
#     return list_of_graphs

In [ ]:
# list_of_graphs = read_graphs_in_path(graph_n20_path)

In [ ]:
# assert len(list_of_graphs) == 6000

In [ ]:
def welsh_powell(graph):
    
    nodes_sorted = sorted(graph.nodes(), key=lambda x: graph.degree(x), reverse=True)
    
    color_map = {}
    
    for node in nodes_sorted:
        neighbor_colors = {color_map[n] for n in graph.neighbors(node) if n in color_map}
        color = next((c for c in range(len(nodes_sorted)) if c not in neighbor_colors), 0)
        color_map[node] = color
    
    chromatic_number = max(color_map.values()) + 1
    return chromatic_number, color_map


# G = nx.Graph()
# G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4), (4, 5), (4, 6)])

# # Calculate chromatic number using Welsh-Powell heuristic
# chromatic_number, color_map = welsh_powell(G)
# print("Chromatic Number:", chromatic_number)
# print("Coloring:", color_map)


In [ ]:
def compute_global_graph_features(path, extension='pickle'):

    graph_description = pd.DataFrame(columns = ["GRAPH_IDX","NUM_NODES","NUM_EDGES","IS_CONNECTED","LOGRATIO_EDGETONODES","DENSITY","CHROMATIC_NUM","NORM_MIS",
                                                "SPECTRAL_GAP","LOG_LARGESTEIGVAL","GIRTH","KEMENY_CONSTANT",
                                                "LOG_SECONDLARGESTEIGVAL","LOG_SMALLESTEIGVAL","MIN_ECC","MAX_ECC","LOGRATIO_MAXMINECC"])

    if os.path.isdir(path):

        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            print(filename)
            if os.path.isfile(file_path):    
                if extension is not None:
                    if filename.endswith(extension):
                        dictionary = pickle.load(open(file_path, 'rb'))

                        current_graph = dictionary['graph']

                        current_graph_num_nodes = current_graph.number_of_nodes() #1
                        current_graph_num_edges = current_graph.number_of_edges() #2
                        current_graph_density = nx.density(current_graph)  #3
                        current_graph_connected = int(nx.is_connected(current_graph)) #4

                        current_graph_logratio_edgestonodes = np.log(current_graph_num_edges/current_graph_num_nodes) #5

                        current_graph_chromatic_num, _ = welsh_powell(current_graph) #6
                        normalized_current_graph_mis = len(list(nx.approximation.maximum_independent_set(current_graph))) / current_graph_num_nodes #7
                        current_graph_assortativity = np.round(nx.degree_assortativity_coefficient(current_graph), 8) #8

                        L = nx.normalized_laplacian_matrix(current_graph)
                        e = np.linalg.eigvals(L.A)
                        e.sort()

                        current_graph_spectral_gap = abs(max(e) - e[len(e)-2]) #9

                        current_graph_loglargesteigenval = np.log(e[len(e)-1]) #10
                        current_graph_logsecondlargesteigenval = np.log(e[len(e)-2]) #11
                        # current_graph_logthirdlargesteigenval = np.log(e[len(e)-3])
                        # current_graph_logfourthlargesteigenval = np.log(e[len(e)-4])
                        # current_graph_logfifthlargesteigenval = np.log(e[len(e)-5])
                        # current_graph_logsixthlargesteigenval = np.log(e[len(e)-6])
                        current_graph_logsmallesteigenval = np.log(e[1]) #Smallest NONTRIVIAL eigenvalue #12
                        
                        try: # eccentricity only works for connected graphs
                            eccs = list(nx.eccentricity(current_graph).values())
                            eccs.sort()
                            current_graph_min_eccentricity = eccs[1] #13
                            current_graph_max_eccentricity = eccs[len(e)-1] #14
                            current_graph_ratio_maxmin_eccentricity = np.log(current_graph_max_eccentricity/current_graph_min_eccentricity) #15
                        except:
                            # if the graph is disconnected then we set all the eccentricity features to zero
                            current_graph_min_eccentricity = 0
                            current_graph_max_eccentricity = 0
                            current_graph_ratio_maxmin_eccentricity = 0
                        
                        current_graph_kemeny_const = np.round(nx.kemeny_constant(current_graph), 8) #16
                        current_graph_girth = nx.girth(current_graph) #17
                        current_graph_transitivity = nx.transitivity(current_graph)
                            
                        # Yash: I have old version of panda but would be nice to use pd.concat instead of append!     
                        graph_description = graph_description.append({"GRAPH_IDX":filename,
                                                                     "NUM_NODES":current_graph_num_nodes,
                                                                     "NUM_EDGES":current_graph_num_edges,
                                                                     "IS_CONNECTED":current_graph_connected, 
                                                                     "LOGRATIO_EDGETONODES":current_graph_logratio_edgestonodes,
                                                                     "DENSITY":current_graph_density,
                                                                     "CHROMATIC_NUM":current_graph_chromatic_num,
                                                                     "NORM_MIS":normalized_current_graph_mis, 
                                                                     "GRAPH_ASSORTATIVITY":current_graph_assortativity,
                                                                     "SPECTRAL_GAP":current_graph_spectral_gap,
                                                                     "LOG_LARGESTEIGVAL":current_graph_loglargesteigenval,
                                                                     "LOG_SECONDLARGESTEIGVAL":current_graph_logsecondlargesteigenval,
                                                                     "LOG_SMALLESTEIGVAL":current_graph_logsmallesteigenval,
                                                                     "MIN_ECC":current_graph_min_eccentricity,
                                                                     "MAX_ECC":current_graph_max_eccentricity,
                                                                     "LOGRATIO_MAXMINECC":current_graph_ratio_maxmin_eccentricity,
                                                                     "GIRTH":current_graph_girth,
                                                                     "KEMENY_CONSTANT":current_graph_kemeny_const,
                                                                     "TRANSITIVITY":current_graph_transitivity}, ignore_index = True) 
                        
                        if '20nodes' in path:
                            graph_desc_path = "global_graph_features_20nodes.csv"
                        elif '100nodes' in path:
                            graph_desc_path = "global_graph_features_100nodes.csv"

                        graph_description.to_csv(f'{graph_desc_path}')

        return graph_description     

In [ ]:
df_20 = compute_global_graph_features(path=graph_n20_path)

In [ ]:
df_100 = compute_graph_features.pylobal_graph_features(path=graph_n100_path)